In [1]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks')
from systems.vault import Vault, RMS
from config import config_dict
import logging
from systems.utils import create_logger
logger = create_logger(log_level=logging.DEBUG, logger_name='datafetcher', print_to_console=True)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Datafeeder starting parameters (right now it'll be hardcoded, and later, it will be fetched from a datafeeder_config variable)
data_inputs = {'1min':{'columns':['open', 'high', 'low', 'close', 'volume,' 'SMA15', 'SMA30'], 'lookback':100}, '1d':{'columns':['open', 'high', 'low', 'close', 'volume,' 'SMA15', 'SMA30'], 'lookback':100}}
tickers = ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'AMZN', 'GOOGL', 'FB', 'NFLX', 'INTC', 'AMD', 'XOM', 'JNJ', 
                                      'JPM', 'V', 'PG', 'UNH', 'DIS', 'HD', 'CRM', 'NKE']

datafeeder_config = {'data_inputs':data_inputs, 'tickers':tickers}

In [3]:
# Create date ranges
date_range_1min = pd.date_range(start='2023-01-01', periods=100, freq='min')
date_range_1d = pd.date_range(start='2023-01-01', periods=200, freq='D')

# Generate random prices and other data
np.random.seed(0)

def generate_data(date_range, granularity):
    data = []
    for date in date_range:
        for ticker in tickers:
            open_price = np.random.uniform(100, 200)
            high_price = open_price + np.random.uniform(0, 10)
            low_price = open_price - np.random.uniform(0, 10)
            close_price = np.random.uniform(low_price, high_price)
            volume = np.random.randint(1000, 10000)
            sma15 = np.random.uniform(100, 200)
            sma30 = np.random.uniform(100, 200)
            data.append([granularity, date, ticker, open_price, high_price, low_price, close_price, volume, sma15, sma30])
    return data

data_1min = generate_data(date_range_1min, '1m')
data_1d = generate_data(date_range_1d, '1d')

# Create DataFrames
columns = ['Granularity', 'Datetime', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'SMA15', 'SMA30']
df_1min = pd.DataFrame(data_1min, columns=columns)
df_1d = pd.DataFrame(data_1d, columns=columns)

# Concatenate DataFrames
df = pd.concat([df_1min, df_1d])

# Set multi-index
df.set_index(['Granularity', 'Datetime', 'Ticker'], inplace=True)

# Pivot the DataFrame to have tickers as columns
df = df.unstack(level='Ticker')
market_data_df = df
market_data_df

Open                          \
Ticker                                 AAPL         AMD        AMZN   
Granularity Datetime                                                  
1d          2023-01-01 00:00:00  116.562493  103.910874  130.585486   
            2023-01-02 00:00:00  162.899126  198.614754  191.992704   
            2023-01-03 00:00:00  152.509741  150.160816  133.251010   
            2023-01-04 00:00:00  108.868452  144.873596  129.972940   
            2023-01-05 00:00:00  104.463011  103.183460  131.745748   
...                                     ...         ...         ...   
1m          2023-01-01 01:35:00  136.166174  129.255988  144.828583   
            2023-01-01 01:36:00  141.638425  159.119038  185.222984   
            2023-01-01 01:37:00  152.324831  174.839357  189.534677   
            2023-01-01 01:38:00  147.957809  156.146723  199.541521   
            2023-01-01 01:39:00  101.380249  180.697320  172.859955   

                                                                     \
Ticker                                  CRM         DIS          FB   
Granularity Datetime                                                  
1d          2023-01-01 00:00:00  163.346655  179.515880  161.362235   
            2023-01-02 00:00:00  194.964674  122.806945  140.407844   
            2023-01-03 00:00:00  170.725334  136.454175  195.658641   
            2023-01-04 00:00:00  110.114188  116.673438  114.007811   
            2023-01-05 00:00:00  155.125233  198.382978  133.497767   
...                                     ...         ...         ...   
1m          2023-01-01 01:35:00  137.983008  118.676003  129.301670   
            2023-01-01 01:36:00  120.772386  143.548348  103.084128   
            2023-01-01 01:37:00  124.525305  170.478076  134.212596   
            2023-01-01 01:38:00  124.331511  136.348448  190.929906   
            2023-01-01 01:39:00  137.525977  192.103204  181.383407   

                                                                     \
Ticker                                GOOGL          HD        INTC   
Granularity Datetime                                                  
1d          2023-01-01 00:00:00  136.619738  170.484681  105.899901   
            2023-01-02 00:00:00  148.180691  165.573005  117.813904   
            2023-01-03 00:00:00  188.009815  106.828705  129.308783   
            2023-01-04 00:00:00  178.216060  178.323143  113.410319   
            2023-01-05 00:00:00  120.299234  124.006643  138.816177   
...                                     ...         ...         ...   
1m          2023-01-01 01:35:00  103.086386  166.090453  107.778196   
            2023-01-01 01:36:00  132.988760  199.205392  160.274325   
            2023-01-01 01:37:00  180.425768  172.305889  181.897518   
            2023-01-01 01:38:00  155.342858  182.274469  121.368278   
            2023-01-01 01:39:00  163.751139  162.637190  146.887898   

                                             ...       SMA30              \
Ticker                                  JNJ  ...         JPM        MSFT   
Granularity Datetime                         ...                           
1d          2023-01-01 00:00:00  159.567237  ...  119.028298  125.187303   
            2023-01-02 00:00:00  160.854017  ...  174.225919  146.160008   
            2023-01-03 00:00:00  160.213986  ...  114.551958  137.153199   
            2023-01-04 00:00:00  127.274067  ...  187.273364  137.444546   
            2023-01-05 00:00:00  103.242375  ...  184.882227  197.927886   
...                                     ...  ...         ...         ...   
1m          2023-01-01 01:35:00  139.236118  ...  112.649078  104.998822   
            2023-01-01 01:36:00  135.690923  ...  139.578363  131.197729   
            2023-01-01 01:37:00  165.610950  ...  154.682940  106.694115   
            2023-01-01 01:38:00  163.419075  ...  121.125119  170.163359   
            2023-01-01 01:39:00  156.448161  ...  111.451201  130.

In [61]:
a = {}
a["12"] = a.get('12', 0) + 2
a

{'12': 2}

In [27]:
len(market_data_df.loc["1m"])

100

In [4]:
vault = Vault(config_dict)
# Load dummy data and check if the signals are generated.
#market_data_df = pd.DataFrame()
signals_output = vault.generate_signals(market_data_df)
logger.debug({'signals_output':signals_output})
rms = RMS(config_dict)
orders = rms.convert_signals_to_orders(signals_output)
logger.debug({'orders':orders})

2024-09-18 00:12:40,261 - datafetcher - DEBUG - {'signals_output': {'signals': [], 'ideal_portfolios': []}} - 966983734.py:5


AttributeError: 'RMS' object has no attribute 'current_portfolio'

In [ ]:
signals_output

In [ ]:
vault.datafeeder_config

In [ ]:
orders

In [ ]:
rms.current_portfolio

In [ ]:
rms.avail_funds_s_1

In [ ]:
rms.avail_funds_s_2

In [ ]:
import json

with open('db/vault/current_portfolio.json') as file:
    current_portfolio = json.load(file)
with open('db/vault/vault.json') as file:
    orders = json.load(file)
with open('db/vault/signals.json') as file:
    signals = json.load(file)
current_portfolio

In [ ]:
orders

In [ ]:
signals